# Find alternative secondary structures for 11ntR variants
Copyright 2023 John Shin under GPL-3.0

Supporting Figures come from this document.

In [1]:
from tqdm import tqdm
import re

import numpy as np

In [2]:
def populateSeqDictFold(filename,seq_dict):
    with open(filename,'r') as f:
        for line in f:
            if line.startswith('>'):

                seq_id,scaffold,seq = line.split('::')
                seq_id = seq_id[1:]
                scaffold = scaffold[3:]
                seq = seq.rstrip()
                
                seq_dict[seq_id] = seq_dict.get(seq_id,{})

                seq_dict[seq_id]['seq'] = seq_dict[seq_id].get('seq',seq)
                
                seq_dict[seq_id][scaffold] = seq_dict[seq_id].get(scaffold,{})
                
            elif line.lower().startswith(('a','u','c','g')):
                seq_dict[seq_id][scaffold]['full_seq'] = line.rstrip()
            
            elif line.startswith(' free energy'):

                free_energy = re.search(r'-\d+.\d+',line).group(0)

                if free_energy:
                    seq_dict[seq_id][scaffold]['dG_ensemble'] = np.float64(free_energy)
                else:
                    print(f"oh no, issue with {seq_id}::{scaffold}")
            elif line.startswith(' frequency'):

                frequency = re.search(r'\d+.\d+',line).group(0)

                if frequency:
                    seq_dict[seq_id][scaffold]['MFE_freq'] = np.float64(frequency)
                else:
                    print(f"oh no, issue with {seq_id}::{scaffold}")


In [3]:
def freqFromDG(dG,dG_ensemble):
    '''Assumes T=37 C and uses R = 1.98717 kcal/mol'''
    return np.exp(-(dG-dG_ensemble)/(1.98717/1000*(37+273.15)))

def populateSeqDictSubopt(filename,seq_dict):
    with open(filename,'r') as f:
                
        for line in f:
            if line.startswith('>'):
                seq_id,scaffold,seq = line.split('::')
                seq_id = seq_id[1:]
                scaffold = scaffold[3:]
                seq = seq.split(' ')[0]
                
                seq_dict[seq_id] = seq_dict.get(seq_id,{})

                seq_dict[seq_id]['seq'] = seq_dict[seq_id].get('seq',seq)
                
                seq_dict[seq_id][scaffold] = seq_dict[seq_id].get(scaffold,{})
                
                alt_structs = 0
                cum_struct_freq = 0
            
            elif re.search(r'^[.()]+',line) and (cum_struct_freq < 0.95):

                free_energy = re.search(r'-\d+.\d+',line)

                if free_energy:
                    free_energy = np.float64(free_energy.group(0))
                    structure = re.search(r'[(.)]+',line).group(0)
                    frequency = freqFromDG(free_energy,
                                   seq_dict[seq_id][scaffold]['dG_ensemble'])
                    
                    seq_dict[seq_id][scaffold][f"alt{alt_structs}"] =\
                        seq_dict[seq_id][scaffold].get(f"alt{alt_structs}",{})
                    
                    seq_dict[seq_id][scaffold][f"alt{alt_structs}"]['free_energy'] = \
                        free_energy
                    seq_dict[seq_id][scaffold][f"alt{alt_structs}"]['structure'] = \
                        structure
                    seq_dict[seq_id][scaffold][f"alt{alt_structs}"]['frequency'] = \
                        frequency
                    
                    cum_struct_freq += frequency
                    
                else:
                    print(f"oh no, issue with {seq_id}::{scaffold}::alt{alt_structs}")
                
                alt_structs += 1
                

In [4]:
data_path = 'Data/structures/'

In [5]:
all_seq_info = {}

populateSeqDictFold(data_path+'all_muts_13854_fold.fasta',all_seq_info)
populateSeqDictFold(data_path+'all_muts_14007_fold.fasta',all_seq_info)
populateSeqDictFold(data_path+'all_muts_14073_fold.fasta',all_seq_info)
populateSeqDictFold(data_path+'all_muts_35311_A_fold.fasta',all_seq_info)
populateSeqDictFold(data_path+'all_muts_35600_fold.fasta',all_seq_info)

In [6]:
populateSeqDictSubopt(data_path+'all_muts_13854_subopt.fasta',all_seq_info)
populateSeqDictSubopt(data_path+'all_muts_14007_subopt.fasta',all_seq_info)
populateSeqDictSubopt(data_path+'all_muts_14073_subopt.fasta',all_seq_info)
populateSeqDictSubopt(data_path+'all_muts_35311_A_subopt.fasta',all_seq_info)
populateSeqDictSubopt(data_path+'all_muts_35600_subopt.fasta',all_seq_info)

In [7]:
scaffold_params = ((11,21),(25,-12))
receptor_params = ((6,11),(-12,-6))

In [8]:
# for k,v in all_seq_info['wt'].items():
#     if k == 'seq':
#         continue
#     print(k)
#     for kk,vv in v.items():
#         if kk.startswith('alt'):
#             print(vv['structure'][slice(*receptor_params[0])],
#                   vv['structure'][slice(*receptor_params[1])],
#                   vv['frequency'])

In [9]:
# wt_counts_dict= {
#       '13854': ('9,9', 0.9522239788764616),
#       '14007': ('10,10', 0.9522239788764599),
#       '14073': ('10,10', 0.9329280855679988),
#       '35311_A': ('10,10', 0.9524190366331612),
#       '35600': ('10,10', 0.9524190366331597)}

# wt_receptors=('(..(())...)',
#               '(.((()).).)',
#               '(..(())...)',
#               '...(())....')

In [10]:
# all_seq_structure = {}
# alt_structures = {}
# alt_receptors = {}
# for receptor_name,receptor_dict in all_seq_info.items():
#     for receptor_k, receptor_v in receptor_dict.items():
#         if receptor_k == 'seq':
#             all_seq_structure[receptor_name] = all_seq_structure.get(receptor_name,{})
#         else:
#             scaffold = receptor_k
#             scaffold_counts_dict = {}
#             receptor_counts_dict = {}
#             for scaffold_k,scaffold_v in receptor_v.items():
#                 if scaffold_k.startswith('alt'):
#                     scaffold_l = scaffold_v['structure'][slice(*scaffold_params[0])].count('(')
#                     scaffold_r = scaffold_v['structure'][slice(*scaffold_params[1])].count(')')
                    
#                     receptor_l = scaffold_v['structure'][slice(
#                                                             *receptor_params[0])]
#                     receptor_r = scaffold_v['structure'][slice(
#                                                             *receptor_params[1])]
                    
#                     freq = scaffold_v['frequency']

#                     scaffold_counts_dict[f"{scaffold_l},{scaffold_r}"] = \
#                                 scaffold_counts_dict.get(f"{scaffold_l},{scaffold_r}",0) + freq
                    
#                     receptor_counts_dict[f"{receptor_l}{receptor_r}"] = \
#                                 receptor_counts_dict.get(f"{receptor_l}{receptor_r}",0) + freq
                    
#             all_seq_structure[receptor_name][scaffold] = scaffold_counts_dict            
            
#             if scaffold_counts_dict.get(wt_counts_dict[scaffold][0]) \
#                                 < wt_counts_dict[scaffold][1]*0.9:
#                 if scaffold == '14073':
#                     if scaffold_counts_dict.get('10,10',0) + scaffold_counts_dict.get('9,9',0) < 0.85:
#                         continue
#                 else:
#                     alt_structures[receptor_name] = alt_structures.get(receptor_name,{})
#                     alt_structures[receptor_name][scaffold] = scaffold_counts_dict
            
#             if sum([receptor_counts_dict.get(idx,0) for idx in wt_receptors]) < 0.75:
#                 alt_receptors[receptor_name] = alt_receptors.get(receptor_name,{})
#                 alt_receptors[receptor_name][scaffold] = receptor_counts_dict
#                 alt_receptors[receptor_name][scaffold]['total'] = sum([receptor_counts_dict.get(idx,0) for idx in wt_receptors])
                


## Synergistic Mutants

In [11]:
import pandas as pd

In [12]:
syn_df = pd.read_csv('../refit_bootstrapped/Figures/GAAA_coop.csv').set_index('seq')
syn_df['muts'] = syn_df[['first_loc','first_res','second_loc','second_res']].apply(
                            (lambda s: ''.join([str(x) for x in s])), axis=1)
# syn_df = syn_df[syn_df['coop']=='S'][['muts','dddG']]
# syn_df.head()

In [13]:
for row in syn_df.iterrows():
    display(all_seq_info[row[1]['muts']])
    break

{'seq': 'UAUGG_AAUAAG',
 '13854': {'full_seq': 'cuaggaUAUGGaacugagucgGGAAcgacugaguuAAUAAGuccuag',
  'dG_ensemble': -21.63,
  'MFE_freq': 0.36245,
  'alt0': {'free_energy': -21.0,
   'structure': '(((((((.((.((((.(((((....))))).))))..)).)))))))',
   'frequency': 0.3598043111443066},
  'alt1': {'free_energy': -20.9,
   'structure': '(((((((....((((.(((((....))))).)))).....)))))))',
   'frequency': 0.30591495325738},
  'alt2': {'free_energy': -19.8,
   'structure': '(((((((.((..(((.(((((....))))).)))...)).)))))))',
   'frequency': 0.05134267383248952},
  'alt3': {'free_energy': -19.5,
   'structure': '(((((((..(.((((.(((((....))))).))))..)..)))))))',
   'frequency': 0.03155596836837083},
  'alt4': {'free_energy': -19.4,
   'structure': '.((((((.((.((((.(((((....))))).))))..)).)))))).',
   'frequency': 0.026829702394893813},
  'alt5': {'free_energy': -19.3,
   'structure': '((((((.....((((.(((((....))))).))))......))))))',
   'frequency': 0.0228113085358546},
  'alt6': {'free_energy': -19.

In [14]:
def fold11ntR(seq,fold,verbose=False,pad=''):
    left = 0
    right = len(seq)-1
    
    lines = []
    
    finished = False
    seen = 0
    
    while not finished:
        
        if verbose:
            print(left,right,seen)
            print('\n'.join(lines[::-1]),'\n')
        
        
        if left == right:
            lines += [f"{pad}{seq[left]}"]
            left += 1
            seen += 1          
        elif fold[left] == '.':            
            if fold[right] == '.':
                lines += [f"{pad}{seq[left]} {seq[right]}"]
                left += 1
                right -= 1
                seen += 2
            else:
                lines += [f"{pad}{seq[left]} |"]
                left += 1
                seen += 1
                
        elif fold[left] == '(':            
            if fold[right] == '.':
                lines += [f"{pad}| {seq[right]}"]
                right -= 1
                seen += 1
            else:
                lines += [f"{pad}{seq[left]}-{seq[right]}"]
                left += 1
                right -= 1
                seen += 2
            
        if seen == len(seq):
            finished = True
            
            
    lines = lines[::-1]
    
    return '\n'.join(lines)
                   

def get11ntRFolds(d,verbose=False):
    
    scaffold_dicts = {}

    for k,v in d.items():        
        if k == 'seq':
            seq = v            
        else:
            scaffold = k
            fold_energies = {}
            fold_freqs = {}
            
            for kk,vv in v.items():
                tot_freq = 0
                if 'alt' in kk:
                    fold = ''.join([vv['structure'][slice(*i)] for i in receptor_params])
                    fold_energies[fold] = fold_energies.get(fold,0) + vv['free_energy']*vv['frequency']
                    fold_freqs[fold] = fold_freqs.get(fold,0) + vv['frequency']
                    
            for k in fold_energies.keys():
                fold_energies[k] = fold_energies[k]/fold_freqs[k]
                
            scaffold_dicts[scaffold] = fold_energies
    
    if verbose:
        return scaffold_dicts
    
    output = {}
    
    for k,v in scaffold_dicts.items():
        fold = sorted(v, key=v.get)[0]
        output[fold] = output.get(fold,{})
        output[fold].update({k:v[fold]})
        
    return output

In [15]:
print('Scaffold 1')
print(fold11ntR('cuaggaUAUGGaacugagucgGGAAcgacugaguuCCUAAGuccuag'.replace('_',''),
                '(((((((..((((((.(((((....))))).))))))...)))))))',pad='   '))

print('Scaffold 2')
print(fold11ntR('cuaggaUAUGGaaugcacaggGGAAccugugcauuCCUAAGuccuag'.replace('_',''),
                '(((((((..((((((((((((....))))))))))))...)))))))',pad='   '))

print('Scaffold 3')
print(fold11ntR('cuaggaUAUGGagggaucuugGGAAcaagaucccuCCUAAGuccuag'.replace('_',''),
                '(((((((..((((((((((((....))))))))))))...)))))))',pad='   '))

print('Scaffold 4')
print(fold11ntR('cuaggaUAUGGaagccggucgGGAAcgaccguggcuuCCUAAGuccuag'.replace('_',''),
                '(((((((..((((((((((((....)))))..)))))))...)))))))',pad='   '))

print('Scaffold 5')
print(fold11ntR('cuaggaUAUGGaagccggucgGGAAcgaccaggcuuCCUAAGuccuag'.replace('_',''),
                '(((((((..((((((((((((....))))).)))))))...)))))))',pad='   '))



Scaffold 1
   G A
   G A
   g-c
   c-g
   u-a
   g-c
   a-u
   g g
   u-a
   c-g
   a-u
   a-u
   G-C
   G-C
   | U
   U A
   A A
   U-G
   a-u
   g-c
   g-c
   a-u
   u-a
   c-g
Scaffold 2
   G A
   G A
   g-c
   g-c
   a-u
   c-g
   a-u
   c-g
   g-c
   u-a
   a-u
   a-u
   G-C
   G-C
   | U
   U A
   A A
   U-G
   a-u
   g-c
   g-c
   a-u
   u-a
   c-g
Scaffold 3
   G A
   G A
   g-c
   u-a
   u-a
   c-g
   u-a
   a-u
   g-c
   g-c
   g-c
   a-u
   G-C
   G-C
   | U
   U A
   A A
   U-G
   a-u
   g-c
   g-c
   a-u
   u-a
   c-g
Scaffold 4
   G A
   G A
   g-c
   c-g
   u-a
   g-c
   g-c
   | g
   | u
   c-g
   c-g
   g-c
   a-u
   a-u
   G-C
   G-C
   | U
   U A
   A A
   U-G
   a-u
   g-c
   g-c
   a-u
   u-a
   c-g
Scaffold 5
   G A
   G A
   g-c
   c-g
   u-a
   g-c
   g-c
   | a
   c-g
   c-g
   g-c
   a-u
   a-u
   G-C
   G-C
   | U
   U A
   A A
   U-G
   a-u
   g-c
   g-c
   a-u
   u-a
   c-g


In [17]:
syn_df['dG_fold'] = [list(get11ntRFolds(all_seq_info[muts]).values())[0]['13854'] for muts in syn_df['muts']]
syn_df['Structure'] = [list(get11ntRFolds(all_seq_info[muts]).keys())[0] for muts in syn_df['muts']]

syn_df[['muts','dddG','dG_fold','Structure']].sort_values('dddG',ascending=False).to_csv('Figures/Supplement/Syn.csv')